In [ ]:
!nvidia-smi

Tue Jan 11 13:05:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#version modifed training less RAM + trivia dataset
!pip install transformers==3.5.1
!pip install torch==1.4.0

     |████████████████████████████████| 1.3 MB 14.0 MB/s 
     |████████████████████████████████| 2.9 MB 81.9 MB/s 
     |████████████████████████████████| 895 kB 90.7 MB/s 
     |████████████████████████████████| 1.1 MB 84.2 MB/s 
     |████████████████████████████████| 753.4 MB 6.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.


In [ ]:
from transformers import BertTokenizer, BertForPreTraining, BertForQuestionAnswering, BertModel, BertConfig
from transformers import XLMRobertaForQuestionAnswering, XLMRobertaTokenizer, XLMRobertaConfig
import torch
import torch.nn as nn
from transformers.data.metrics.squad_metrics import compute_predictions_log_probs, compute_predictions_logits, squad_evaluate
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

In [ ]:
# class mBert(nn.Module):
#     def __init__(self):
#         super(mBert, self).__init__()
#         self.bert_block = BertModel.from_pretrained("bert-base-multilingual-cased")
#         self.question = nn.Linear(768, 2, bias=True)
#     def forward(self, inputs):
#         pooled_output, sequence_output = self.bert_block(inputs)
#         return self.question(sequence_output)

In [ ]:
# model = XLMRobertaForQuestionAnswering.from_pretrained('../input/ckpt-xlmr-xquad-pretrain/final_model')
model = XLMRobertaForQuestionAnswering.from_pretrained('xlm-roberta-large')

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream t

In [ ]:
# model = mBert()
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

In [ ]:
# !mkdir dataset \
# && cd dataset \
# && wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json \
# && wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

In [ ]:
processor = SquadV1Processor()

In [ ]:
# train_examples = processor.get_train_examples('../input/trivia-full-filted','train_wiki_total.json')
# dev_examples = processor.get_dev_examples('../input/viquad-v1','dev_ViQuAD.json')

In [ ]:
from transformers.data.processors.squad import squad_convert_examples_to_features

In [ ]:
def read_saved_data(input_dir,evaluate=False,output_examples=False):
    from torch.utils.data import TensorDataset

    if output_examples:
        feat="features;datasets;examples"
    else:
        feat="datasets"

    all_features = {"features":[],"examples":[],"datasets":[]}
    all_input_ids = torch.tensor([], dtype=torch.long)
    all_attention_masks = torch.tensor([], dtype=torch.long)
    all_token_type_ids = torch.tensor([], dtype=torch.long)
    all_cls_index = torch.tensor([], dtype=torch.long)
    all_p_mask = torch.tensor([], dtype=torch.float)
    all_is_impossible = torch.tensor([], dtype=torch.float)
    all_start_positions = torch.tensor([], dtype=torch.long)
    all_end_positions = torch.tensor([], dtype=torch.long)

    for i in feat.split(";"):
        for file_name in os.listdir(os.path.join(input_dir,i)):
            data = torch.load(os.path.join(input_dir,i,file_name))[i]
            if isinstance(data,TensorDataset):
                if evaluate:
                    all_input_ids = torch.cat([all_input_ids,data.tensors[0]],dim=0)
                    all_attention_masks = torch.cat([all_attention_masks,data.tensors[1]],dim=0)
                    all_token_type_ids = torch.cat([all_token_type_ids,data.tensors[2]],dim=0)
                    all_cls_index = torch.cat([all_cls_index,data.tensors[4]],dim=0)
                    all_p_mask = torch.cat([all_p_mask,data.tensors[5]],dim=0)
                else:
                    all_input_ids = torch.cat([all_input_ids,data.tensors[0]],dim=0)
                    all_attention_masks = torch.cat([all_attention_masks,data.tensors[1]],dim=0)
                    all_token_type_ids = torch.cat([all_token_type_ids,data.tensors[2]],dim=0)
                    all_start_positions = torch.cat([all_start_positions,data.tensors[3]],dim=0)
                    all_end_positions = torch.cat([all_end_positions,data.tensors[4]],dim=0)
                    all_cls_index = torch.cat([all_cls_index,data.tensors[5]],dim=0)
                    all_p_mask = torch.cat([all_p_mask,data.tensors[6]],dim=0)
                    all_is_impossible = torch.cat([all_is_impossible,data.tensors[7]],dim=0)
            elif isinstance(data,list):
                all_features[i] += data
    
    if evaluate and "datasets" in feat.split(";"):
        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        all_features["datasets"] = TensorDataset(all_input_ids, all_attention_masks, all_token_type_ids, all_example_index, all_cls_index, all_p_mask)
    elif not evaluate and "datasets" in feat.split(";"):
        all_features["datasets"] = TensorDataset(all_input_ids,all_attention_masks,all_token_type_ids,all_start_positions,all_end_positions,all_cls_index,all_p_mask,all_is_impossible,)


    if output_examples:
        return all_features['datasets'], all_features['examples'], all_features['features']
    else:
        return all_features['datasets']

In [ ]:
import gc
max_seq_length = 384
def load_and_cache_examples(tokenizer, evaluate=False, output_examples= False):
    input_dir = "."
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}_{}_{}".format(
            "dev" if evaluate else "train",
            "xlmr",
            str(max_seq_length),
        ),
    )
    root_dir = os.path.join(cached_features_file+"_dir")
    features_file = os.path.join(root_dir,'features')
    datasets_file = os.path.join(root_dir,'datasets')
    examples_file = os.path.join(root_dir,'examples')
    if evaluate:
        print("NO EVAL")
#         examples = processor.get_dev_examples('../input/viquad-v1', 'dev_ViQuAD.json')
#         examples = processor.get_dev_examples('../input/merge-dataset', 'merge_dataset_v1_dev.json')
    else:
#         examples = processor.get_train_examples('../input/viquad-v1', 'train_ViQuAD.json')
        examples = processor.get_train_examples('/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Dataset','train_newqa_total.json')
    for i,j in enumerate(range(0, len(examples), 20000)):
        sub_examples = examples[j:j+20000]
        features, dataset = squad_convert_examples_to_features(
                examples=sub_examples,
                tokenizer=tokenizer,
                max_seq_length=384,
                doc_stride=128,
                max_query_length=64,
                is_training=not evaluate,
                return_dataset="pt",
                threads=10,
            )
        if not os.path.exists(os.path.join(features_file)):
            os.makedirs(os.path.join(features_file))
        if not os.path.exists(os.path.join(datasets_file)):
            os.makedirs(os.path.join(datasets_file))
        if not os.path.exists(os.path.join(examples_file)):
            os.makedirs(os.path.join(examples_file))
        print("Saving features into cached files %s, %s, %s", os.path.join(features_file,'features_'+str(i)),os.path.join(datasets_file,'datasets_'+str(i)),os.path.join(examples_file,'examples_'+str(i)))
        torch.save({"features": features}, os.path.join(features_file,'features_'+str(i)))
        torch.save({"datasets": dataset}, os.path.join(datasets_file,'datasets_'+str(i)))
        torch.save({"examples": sub_examples}, os.path.join(examples_file,'examples_'+str(i)))
        print("Done")
        del features, dataset
        gc.collect()
    del examples
    return read_saved_data(root_dir,evaluate=evaluate,output_examples=output_examples)

In [ ]:
import os
train_dataset = load_and_cache_examples(tokenizer, evaluate= False, output_examples = False)

100%|██████████| 40805/40805 [00:50<00:00, 801.43it/s]
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
convert squad examples to features:   0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in

Saving features into cached files %s, %s, %s ./cached_train_xlmr_384_dir/features/features_0 ./cached_train_xlmr_384_dir/datasets/datasets_0 ./cached_train_xlmr_384_dir/examples/examples_0
Done


convert squad examples to features:   0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  Fu

Saving features into cached files %s, %s, %s ./cached_train_xlmr_384_dir/features/features_1 ./cached_train_xlmr_384_dir/datasets/datasets_1 ./cached_train_xlmr_384_dir/examples/examples_1
Done


convert squad examples to features:   0%|          | 0/805 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  Futu

Saving features into cached files %s, %s, %s ./cached_train_xlmr_384_dir/features/features_2 ./cached_train_xlmr_384_dir/datasets/datasets_2 ./cached_train_xlmr_384_dir/examples/examples_2
Done


In [ ]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [ ]:
import os
def evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features):
    eval_sampler = SequentialSampler(dev_dataset)
    eval_dataloader = DataLoader(dev_dataset, sampler=eval_sampler, batch_size=12)
    all_results = []
#     start_time = timeit.default_timer()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            del inputs["token_type_ids"]
            example_indices = batch[3]
            outputs = model(**inputs)
        for i, example_index in enumerate(example_indices):
            eval_feature = dev_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
#             for output in outputs:
#                 print(output)
            output = [to_list(output[i]) for output in outputs]
#             output = [to_list(output) for output in outputs]
            if len(output) >= 5:
                start_logits = output[0]
                start_top_index = output[1]
                end_logits = output[2]
                end_top_index = output[3]
                cls_logits = output[4]

                result = SquadResult(
                    unique_id,
                    start_logits,
                    end_logits,
                    start_top_index=start_top_index,
                    end_top_index=end_top_index,
                    cls_logits=cls_logits,
                )
            else:
                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
            all_results.append(result)
    
    output_prediction_file = os.path.join("./", "predictions_{}.json".format(""))
    output_nbest_file = os.path.join("./", "nbest_predictions_{}.json".format(""))
    output_null_log_odds_file = os.path.join("./", "null_odds_{}.json".format(""))
    predictions = compute_predictions_logits(
            dev_examples,
            dev_features,
            all_results,
            20,
            300,
            False,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            True,
            False,
            0.0,
            tokenizer,
        )
    results = squad_evaluate(dev_examples, predictions)
    return results

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import trange, tqdm
device = torch.device('cuda')

In [ ]:
# for param in model.bert.parameters():
#     param.requires_grad = False

In [15]:
import numpy as np
num_epochs = 1
tb_writer = SummaryWriter()
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=4)
t_total = len(train_dataloader) // 1 * num_epochs


no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=814, num_training_steps=t_total
)

device = torch.device('cuda')

model.to(device)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0

model.zero_grad()
train_iterator = trange(
    epochs_trained, int(num_epochs), desc="Epoch", disable=-1 not in [-1, 0]
)

from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
    for step, batch in enumerate(epoch_iterator):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
        del inputs["token_type_ids"]
        outputs = model(**inputs)
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        global_step += 1

        if global_step % 5000 == 0:
#             output_dir = os.path.join('./', "checkpoint-{}".format(global_step))
#             model_to_save = model.module if hasattr(model, "module") else model
#             model_to_save.save_pretrained(output_dir)
#             tokenizer.save_pretrained(output_dir)
#             torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
#             torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
            print(" global_step = %s, average loss = %s", global_step, tr_loss/global_step)

            
output_dir = os.path.join('/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Data Preprocessing', 'final_model_newqa_Jan11')
model_to_save = model.module if hasattr(model, "module") else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# print(" global_step = %s, average loss = %s", global_step, tr_loss/global_step)

# results = evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features)
# for key, value in results.items():
#     print(key, value)

Iteration:  15%|█▍        | 4999/33388 [53:11<5:01:49,  1.57it/s]

 global_step = %s, average loss = %s 5000 1.8118483253791928


Iteration:  30%|██▉       | 9999/33388 [1:46:24<4:08:36,  1.57it/s]

 global_step = %s, average loss = %s 10000 1.5698463851884008


Iteration:  45%|████▍     | 14999/33388 [2:39:37<3:15:44,  1.57it/s]

...

 global_step = %s, average loss = %s 25000 1.3209817822427303


Iteration:  90%|████████▉ | 29999/33388 [5:19:08<36:05,  1.57it/s]

 global_step = %s, average loss = %s 30000 1.2676217542879593


Epoch: 100%|██████████| 1/1 [5:55:10<00:00, 21310.09s/it]


('/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Data Preprocessing/final_model_newqa_Jan11/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Data Preprocessing/final_model_newqa_Jan11/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Data Preprocessing/final_model_newqa_Jan11/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Data Preprocessing/final_model_newqa_Jan11/added_tokens.json')

In [ ]:
# tokenizer_1 = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")

In [ ]:
# test_examples = processor.get_dev_examples('../input/viquad-v1','test_ViQuAD.json')
# test_features, test_dataset = squad_convert_examples_to_features(test_examples, 
#                                                        tokenizer, 
#                                                        max_seq_length = 384, 
#                                                        doc_stride = 128,
#                                                        max_query_length = 64,
#                                                        is_training = False,
#                                                        return_dataset = 'pt',
#                                                        threads = 10
#                                                        )

In [ ]:
# results = evaluate(model, tokenizer_1, test_dataset, test_examples, test_features)
# for key, value in results.items():
#     print(key, value)